In [2]:
from models.piece import Piece
from models.board import Board

In [3]:
class Game:
    """A game is similar to a problem, but it has a terminal test instead of 
    a goal test, and a utility for each terminal state. To create a game, 
    subclass this class and implement `actions`, `result`, `is_terminal`, 
    and `utility`. You will also need to set the .initial attribute to the 
    initial state; this can be done in the constructor."""

    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)
    
    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError
        

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose: 
            print('Player', player, 'move:', move)
            print(state)
    return state

In [4]:
# Develop a player model and implement search algorithm 

# Search algorithm: Minimax with Alpha-Beta Pruning
def minimax_search(game, state, depth=3, maximizing_player=True):
    def minimax(state, depth, alpha, beta, maximizing_player):
        if game.is_terminal(state) or depth == 0:
            return game.utility(state, state.to_move)

        if maximizing_player:
            max_eval = float('-inf')
            for move in game.actions(state):
                eval = minimax(game.result(state, move), depth-1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break  # Prune
            return max_eval
        else:
            min_eval = float('inf')
            for move in game.actions(state):
                eval = minimax(game.result(state, move), depth-1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break  # Prune
            return min_eval

    # Choose the best move
    best_score = float('-inf')
    best_move = None
    for move in game.actions(state):
        score = minimax(game.result(state, move), depth, float('-inf'), float('inf'), maximizing_player)
        if score > best_score:
            best_score = score
            best_move = move
    return best_move
    
# Player model that uses the search algorithm
def minimax_player(game, state):
    return minimax_search(game, state)

In [5]:
class Checkers(Game):
    def __init__(self, players):
        """
        players: a list of two identifiers (e.g., ["White", "Black"])
        The state is a dict with keys:
          - 'board': an instance of Board
          - 'turn': whose turn it is to move
        """
        self.players = players
        self.initial = self.create_initial_state()
    
    def create_initial_state(self):
        board = Board()
        board.setup(self.players)
        # Set initial turn to players[0]
        return {"board": board, "turn": self.players[0]}
    
    def actions(self, state):
        """
        Return a list of valid moves for the current state's turn.
        Each move is represented as a tuple: ((from_row, from_col), (to_row, to_col)).
        For simplicity, multi-jump moves are not implemented.
        """
        valid_moves = []
        board = state["board"].grid
        current = state["turn"]
        for row in range(8):
            for col in range(8):
                piece = board[row][col]
                if piece is not None and piece.player == current:
                    moves = self.get_valid_moves(board, (row, col), piece)
                    for move in moves:
                        valid_moves.append(((row, col), move))
        return valid_moves
    
    def get_valid_moves(self, board, pos, piece):
        """
        Return valid destination positions for the piece at pos.
        It uses the piece's available_moves method and then checks the board.
        If the destination square is occupied by an opponent, a jump move is considered.
        """
        moves = []
        # Get candidate moves from the piece.
        for candidate in piece.available_moves():
            new_row, new_col = candidate[1], candidate[2]
            if board[new_row][new_col] is None:
                moves.append((new_row, new_col))
            else:
                # If the adjacent square has an opponent piece, try a jump move.
                if board[new_row][new_col].player != piece.player:
                    jump_candidate = piece.jump_move(candidate)
                    if jump_candidate is not None:
                        jump_row, jump_col = jump_candidate[1], jump_candidate[2]
                        if board[jump_row][jump_col] is None:
                            moves.append((jump_row, jump_col))
        return moves
    
    def result(self, state, move):
        """
        Given a state and a move ((from_row, from_col), (to_row, to_col)),
        return a new state with the move executed.
        This includes moving the piece, capturing any jumped piece,
        promoting the piece (if applicable), and switching the turn.
        """
        new_state = copy.deepcopy(state)
        board_obj = new_state["board"]
        board = board_obj.grid
        (from_row, from_col), (to_row, to_col) = move
        piece = board[from_row][from_col]
        
        # Move the piece using its own move method.
        piece.move((to_row, to_col))
        board[to_row][to_col] = piece
        board[from_row][from_col] = None

        # Handle capturing: if the move spans two rows and columns, remove the jumped piece.
        if abs(from_row - to_row) == 2 and abs(from_col - to_col) == 2:
            mid_row = (from_row + to_row) // 2
            mid_col = (from_col + to_col) // 2
            board[mid_row][mid_col] = None

        # Switch turn.
        new_state["turn"] = self.players[1] if state["turn"] == self.players[0] else self.players[0]
        return new_state

    def is_terminal(self, state):
        """
        The game is terminal if one player has no pieces left or if the current
        player has no valid moves.
        """
        board = state["board"].grid
        pieces_count = {self.players[0]: 0, self.players[1]: 0}
        for row in board:
            for cell in row:
                if cell is not None:
                    pieces_count[cell.player] += 1
        if pieces_count[self.players[0]] == 0 or pieces_count[self.players[1]] == 0:
            return True
        
        # Terminal if current player has no moves.
        if not self.actions(state):
            return True
        
        return False
    
    def utility(self, state, player):
        """
        Return 1 if the given player wins, -1 if the player loses,
        and 0 for a tie or non-terminal state.
        This assumes that the state is terminal.
        """
        if not self.is_terminal(state):
            return 0
        board = state["board"].grid
        pieces_count = {self.players[0]: 0, self.players[1]: 0}
        for row in board:
            for cell in row:
                if cell is not None:
                    pieces_count[cell.player] += 1
        
        if pieces_count[self.players[0]] == 0:
            return 1 if player == self.players[1] else -1
        if pieces_count[self.players[1]] == 0:
            return 1 if player == self.players[0] else -1
        
        # If the game ended due to no moves, consider it a loss for the player who cannot move.
        if not self.actions(state):
            return -1
        
        return 0